# This program is use for automatic youtube subtitles translator using bard
## I mostly use online libaries for this but atleast it's working

## Requirement
## python, pip
### pip installation tutorial: " https://www.geeksforgeeks.org/how-to-install-pip-on-windows/ "


In [5]:
! pip install getenv
! pip install pytube
! pip install youtube_transcript_api
! pip install bardapi

  Using cached bardapi-0.1.27-py3-none-any.whl (22 kB)
  Using cached google_cloud_translate-2.0.1-py2.py3-none-any.whl (90 kB)
  Using cached deep_translator-1.11.4-py3-none-any.whl (42 kB)
  Using cached browser_cookie3-0.19.1-py3-none-any.whl (14 kB)
  Using cached httpx-0.24.1-py3-none-any.whl (75 kB)
  Using cached google_cloud_core-1.7.3-py2.py3-none-any.whl (28 kB)
  Using cached google_api_core-1.34.0-py3-none-any.whl (120 kB)
  Using cached pycryptodomex-3.18.0-cp35-abi3-win_amd64.whl (1.7 MB)
  Using cached lz4-4.3.2-cp39-cp39-win_amd64.whl (99 kB)
  Using cached httpcore-0.17.3-py3-none-any.whl (74 kB)
  Using cached h2-4.1.0-py3-none-any.whl (57 kB)
  Using cached google_auth-2.22.0-py2.py3-none-any.whl (181 kB)
  Using cached protobuf-3.20.3-cp39-cp39-win_amd64.whl (904 kB)
  Using cached googleapis_common_protos-1.59.1-py2.py3-none-any.whl (224 kB)
  Using cached grpcio_status-1.56.2-py3-none-any.whl (5.1 kB)
  Using cached grpcio-1.56.2-cp39-cp39-win_amd64.whl (4.2 MB)
 

In [1]:
import pytube
from youtube_transcript_api import YouTubeTranscriptApi
from bardapi import Bard
import os
import requests

In [6]:
#đưa link video vào đây
link = "https://www.youtube.com/watch?v=CRrf3h9vhp8"
source = link + "?cc_load_policy=1"
yt = pytube.YouTube(source)

#tách link ra để lấy video id
id = source.split("v=", 2)[-1]
#check các loại phụ đề đã có trước hoặc khi video tắt phụ đề
cc = YouTubeTranscriptApi.list_transcripts(id)
print("id: ", id, "\n")
print("available language: ")
for i, df in enumerate(cc):
    print(df)


id:  CRrf3h9vhp8?cc_load_policy=1 

available language: 
zh-HK ("Chinese (Hong Kong) - Chinese - Hong Kong SAR")[TRANSLATABLE]
zh-Hant ("Chinese (Traditional) - Chinese (Traditional)")[TRANSLATABLE]
nl ("Dutch - Dutch")[TRANSLATABLE]
en ("English - English")[TRANSLATABLE]
fr ("French - French")[TRANSLATABLE]
de ("German - German")[TRANSLATABLE]
it ("Italian - Italian")[TRANSLATABLE]
ja ("Japanese - Japanese")[TRANSLATABLE]
ko ("Korean - Korean")[TRANSLATABLE]
pl ("Polish - Polish")[TRANSLATABLE]
pt ("Portuguese - Portuguese")[TRANSLATABLE]
ru ("Russian - Russian")[TRANSLATABLE]
es ("Spanish - Spanish")[TRANSLATABLE]
tr ("Turkish - Turkish")[TRANSLATABLE]


In [7]:
#ngôn ngữ muốn đọc phụ đề
lang = 'en'
#lấy phụ đề về và để trong transcript
transcript = YouTubeTranscriptApi.get_transcript(id,languages=[lang, 'vi'])
#ghi phụ đề vào lyrics.txt
with open("lyrics.txt", "w", encoding = 'utf-8') as f:
    for subtitle in transcript:
        text = subtitle['text']
        start = subtitle['start']
        duration = subtitle['duration']
        f.write(f"{text}\n") #nếu muốn ghi cả timeline có thể thêm {start} -> {start+duration} nhưng sẽ thường bị lỗi khi dịch
f.close()


In [8]:
#mở file để đọc
f = open("lyrics.txt", "r")

''' this is for an emergency
cookie_dict = {
    token = 'YQiLtpIxwMX_V3gqKce011fmUyY9L_KCvlyUITEhuhTLUa7RmNPGUwC8ta-nvEhwfuOi6A.'
    "__Secure-1PSID": "YAitmuOXowTFRVB33nbZlL5D4ps2IWYIQDVoY9E0qk3lk5OhoeRC_9bzovpfIOX0st0ANQ.",
    "__Secure-1PSIDTS": "sidts-CjIBPu3jIc1Yu8axRx1tKR6-pJzF2iFi2TU3Xp7jysMJHdzdiLBYH3lFMSmlwYT4-zd9qBAA",
    # Any cookie values you want to pass session object.
}
'''
#đưa token vào để sd(giống Chat gpt key)
os.environ['_BARD_API_KEY'] = 'YQiLtpIxwMX_V3gqKce011fmUyY9L_KCvlyUITEhuhTLUa7RmNPGUwC8ta-nvEhwfuOi6A.'
token = 'YQiLtpIxwMX_V3gqKce011fmUyY9L_KCvlyUITEhuhTLUa7RmNPGUwC8ta-nvEhwfuOi6A.'
#yêu cầu session sử dụng cookies để lưu trữ đoạn chat cũ
session = requests.Session()
session.headers = {
            "Host": "bard.google.com",
            "X-Same-Domain": "1",
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.114 Safari/537.36 OPR/88.0.4390.60",
            "Content-Type": "application/x-www-form-urlencoded;charset=UTF-8",
            "Origin": "https://bard.google.com",
            "Referer": "https://bard.google.com/",
        }
session.cookies.set("__Secure-1PSID", token)  
#khởi tạo bard
bard = Bard(token=token, session=session)

#Nếu muốn dịch sang ngôn ngữ nào hãy sử dụng chính ngôn ngữ đấy để yêu cầu bard dịch
bard.get_answer("dịch giúp tôi một đoạn văn sang tiếng việt được không")
#đưa đoạn văn cho bard
ans = bard.get_answer(f.read())['content']
#đóng file
f.close()
#in kết quả trả về
print(ans)



Sure, here is the translation of the lyrics of "Traitor" by Olivia Rodrigo in Vietnamese:

* * *

Ooh, ooh, ooh
Ah, ah, ah, ah

Mắt nâu tội lỗi và những lời nói dối trắng trợn
Vâng, tôi giả vờ ngu ngốc nhưng tôi luôn biết
Rằng bạn đã nói chuyện với cô ấy, có thể
Đã làm điều tồi tệ hơn
Tôi giữ im lặng
Để tôi có thể giữ bạn

Và thật buồn cười
Làm sao bạn lại chạy đến với cô ấy
Thứ hai chúng tôi gọi là kết thúc

Và thật buồn cười
Làm sao bạn lại nói rằng bạn là bạn bè
Bây giờ nó chắc chắn không giống như vậy

Bạn đã phản bội tôi
Và tôi biết rằng bạn sẽ
Không bao giờ cảm thấy hối tiếc
Vì cách tôi đã đau, vâng

Bạn đã nói chuyện với cô ấy
Khi chúng ta ở bên nhau
Yêu bạn trong thời điểm tồi tệ nhất
Nhưng điều đó không thành vấn đề
Phải mất hai tuần
Để đi và hẹn hò với cô ấy
Chắc bạn không lừa dối
Nhưng bạn vẫn là kẻ phản bội

Bây giờ bạn mang cô ấy đi vòng quanh
Chỉ để dập tắt tôi
Hiển thị cô ấy
Như cô ấy là một chiếc cúp mới

Và tôi biết nếu bạn là người thật
Không có cách nào bạn có thể
Yê

In [ ]:
#bard is actually shit i don't really recommend this for translation.